In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import imageio
import os
from ipywidgets import widgets

In [39]:
# Load in gain_data.csv, first column is the index
gain_data = pd.read_csv('gain_data.csv', index_col=0)
gain_data

,Kp,Ki,Kd,RMSE
0,210,1,1,0.581014
1,210,1,2,0.471163
2,210,1,3,0.417737
3,210,1,4,0.388121
4,210,1,5,0.368037
...,...,...,...,...
7595,390,20,16,0.161619
7596,390,20,17,0.159330
7597,390,20,18,0.157274
7598,390,20,19,0.155425


In [3]:
# Initialise seaborn
sns.set_style(style="whitegrid")

In [34]:
gain_data.shape

(7600, 4)

In [4]:
# Get the RMSE min and max values
RMSE_min = gain_data['RMSE'].min()
RMSE_max = gain_data['RMSE'].max()

In [90]:
# Generalize above function to plot any 3 variables
def plot_3d_plane(data, x, y, z, title, filename, flip_x=False, flip_y=False, flip_z=False):
    RMSE_min = data['RMSE'].min()
    RMSE_max = data['RMSE'].max()
    
    with imageio.get_writer(filename + ".gif", mode='I', duration=0.15) as writer:
        
        for i, x_val in enumerate(data[x].unique()):
            # Filter data for current x
            data_x = data[data[x] == x_val]
                
            # Create figure
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(111, projection='3d')
            

            # Set RMSE limits
            ax.set_zlim(RMSE_min, RMSE_max)
            
        
            # Plot data as a plane
            # ax.plot_trisurf(data_x[y] if not flip_y else data_x[y][::-1],
            #                 data_x[z] if not flip_z else data_x[z][::-1],
            #                 data_x['RMSE'], cmap='viridis', edgecolors='none', shade="gouraud", antialiased=False, linewidth=0.0)
            
                  
            # Increase the number of points to make the surface smoother via interpolation
            data_interp = data_x.interpolate(method='cubic', limit_direction='both', limit=10)
            
            # Rewrite the above to make the colors consistent (use RMSE_max as actual max color)
            ax.plot_trisurf(data_interp[y] if not flip_y else data_interp[y][::-1],
                            data_interp[z] if not flip_z else data_interp[z][::-1],
                            data_interp['RMSE'], edgecolors='none', cmap="rocket_r", shade=True, antialiased=False, linewidth=0.0, vmin=RMSE_min, vmax=RMSE_max)                             
            
            # Set labels
            ax.set_xlabel(y)
            ax.set_ylabel(z)
            ax.set_zlabel('RMSE')
            ax.set_title('{} = {}'.format(x, x_val))
            
            # Change view angle based on current index
            # ax.view_init(elev=30, azim=10 + i * 2)
            
            # Make graph fit imagesize
            plt.tight_layout()
            
            # Save figure
            fig.savefig('{}_{}.png'.format(filename, x_val))
            # Add figure to GIF
            writer.append_data(imageio.imread('{}_{}.png'.format(filename, x_val)))
            # Close figure
            plt.close(fig)
            
        # Loop over the generated images in reverse and append them to the GIF
        for x_val in data[x].unique()[::-1]:
            writer.append_data(imageio.imread('{}_{}.png'.format(filename, x_val)))
            
    # Clear all temp generated files
    # for file in os.listdir():
    #     if file.startswith('{}_'.format(filename)) and file.endswith('.png'):
    #         os.remove(file)
    
    # Return GIF path
    return filename + '.gif'

In [91]:
# Plot Kd and Kp with Ki sliding
gif = plot_3d_plane(gain_data, 'Ki', 'Kd', 'Kp', 'Ki', 'Ki_Kd_Kp', flip_x=False, flip_y=False, flip_z=True)
# Show GIF in notebook
with open(gif,'rb') as file:
    display(widgets.Image(value=file.read(), format='png'))

Image(value=b'GIF89a\xd0\x02\xd0\x02\x87\x00\x00\xff\xff\xff\xfe\xfe\xfe\xfd\xfd\xfd\xfc\xfc\xfc\xfb\xfb\xfb\x…

In [92]:
# Plot Kp and Ki with Kd sliding
gif = plot_3d_plane(gain_data, 'Kd', 'Kp', 'Ki', 'Kd', 'Kd_Kp_Ki', flip_x=False, flip_y=False, flip_z=False)
# Show GIF in notebook
with open(gif,'rb') as file:
    display(widgets.Image(value=file.read(), format='png'))

Image(value=b'GIF89a\xd0\x02\xd0\x02\x87\x00\x00\xff\xff\xff\xfe\xfe\xfe\xfd\xfd\xfd\xfc\xfc\xfc\xfb\xfb\xfb\x…

In [93]:
# Plot Ki and Kd with Kp sliding
gif = plot_3d_plane(gain_data, 'Kp', 'Ki', 'Kd', 'Kp', 'Kp_Ki_Kd', flip_x=False, flip_y=True, flip_z=True)
# Show GIF in notebook
with open(gif,'rb') as file:
    display(widgets.Image(value=file.read(), format='png'))

Image(value=b'GIF89a\xd0\x02\xd0\x02\x87\x00\x00\xff\xff\xff\xfe\xfe\xfe\xfd\xfd\xfd\xfc\xfc\xfc\xfb\xfb\xfb\x…